In [1]:
import pygame
import numpy as np

def polarToCart(rad, thet):
    x = rad * np.cos(thet)
    y = rad * np.sin(thet)
    return (x, y)

def cartToPolar(x, y):
    rad = np.sqrt(x**2 + y**2)
    thet = np.arctan2(y, x)
    return (rad, thet)

def mod2Pi(thet):
    return (thet + 2*np.pi) % (2*np.pi)

pygame 2.3.0 (SDL 2.24.2, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
testDash = "--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------"
testHash = "########################################################################################################################################################################################################"
testHash = testHash + testHash + testHash + testHash
testDash = testDash + testDash + testDash + testDash

with open("sim.txt", "r") as f:
    testSim = f.read()
    testSim = testSim.replace("\n", " ")
    testSim = testSim.replace("  ", " ")

print(testSim)

Summary TL;DR: Self-supervised learning may create AGI or its foundation. What would that look like? Unlike the limit of RL, the limit of self-supervised learning has received surprisingly little conceptual attention, and recent progress has made deconfusion in this domain more pressing. Existing AI taxonomies either fail to capture important properties of self-supervised models or lead to confusing propositions. For instance, GPT policies do not seem globally agentic, yet can be conditioned to behave in goal-directed ways. This post describes a frame that enables more natural reasoning about properties like agency: GPT, insofar as it is inner-aligned, is a simulator which can simulate agentic and non-agentic simulacra. The purpose of this post is to capture these objects in words so GPT can reference them and provide a better foundation for understanding them. I use the generic term “simulator” to refer to models trained with predictive loss on a self-supervised dataset, invariant to 

In [3]:
testStr = testSim
title = "Simulators"

In [4]:
# display string
debug = False
pygame.init()
dimX = 720
dimY = 720
screen = pygame.display.set_mode((dimX, dimY))
clock = pygame.time.Clock()
running = True
font = pygame.font.SysFont("Consolas", 200)

upTheta = 3 * np.pi / 2
baseRad = 75

radius = baseRad * 2
theta = upTheta

radStep = 1.007
stepsPerCircle = 40
thetaStep = 2 * np.pi / stepsPerCircle

frameRate = 144

scaleSpeed = 1/30
rotSpeed = 1/3


while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    screen.fill("white")

    keys = pygame.key.get_pressed()

    fps = clock.get_fps()
    if debug:
        print(fps)
    if fps == 0:
        fps = 15

    # move text
    if keys[pygame.K_UP]:
        radius *= radStep ** ((stepsPerCircle * scaleSpeed) / (fps/15))
    if keys[pygame.K_DOWN]:
        radius /= radStep ** ((stepsPerCircle * scaleSpeed) / (fps/15))
    if keys[pygame.K_LEFT]:
        theta += thetaStep * (rotSpeed / (fps/15))
        radius *= radStep ** (rotSpeed / (fps/15))
    if keys[pygame.K_RIGHT]:
        theta -= thetaStep * (rotSpeed / (fps/15))
        radius /= radStep ** (rotSpeed / (fps/15))

    # move speed
    if keys[pygame.K_w]:
        scaleSpeed *= 1.1
    if keys[pygame.K_s]:
        scaleSpeed /= 1.1
    if keys[pygame.K_a]:
        rotSpeed *= 1.1
    if keys[pygame.K_d]:
        rotSpeed /= 1.1

    titleText = font.render(title, True, "black")
    titleText = pygame.transform.scale_by(titleText, (radius * 1.75)/titleText.get_width())
    titleRect = titleText.get_rect()
    titleRect.center = (dimX/2, dimY/2)
    screen.blit(titleText, titleRect)
    
    r, t = radius, theta
    for i in range(len(testStr)):
        if r < baseRad * (2/3): # old small text
            r *= radStep
            t += thetaStep
            continue
        if r > np.sqrt((dimX/2)**2 + (dimY/2)**2): # outside of screen
            break
        text = font.render(testStr[i], True, "black")
        x, y = polarToCart(r, t)
        
        newScale = 200 * (r / baseRad) / 10
        x += dimX/2 
        y += dimY/2

        # draw dot at x, y
        if debug:
            if x > 0 and x < dimX and y > 0 and y < dimY:
                pygame.draw.circle(screen, "red", (x, y), 3, 3)

        # left right -0.5, -0.3
        # up down -0.3 -0.5
        # both diag -0.55, -0.55
        
        # this is to adjust the location of the rotated text to match the spiral
        # when adding the title copilot just suggested to center the rectangle, so didn't need all of this but leaving it anyways
        u = -0.5
        l = -0.28
        tmod = mod2Pi(t)
        # down
        if tmod >= np.pi / 4 and tmod <= 3 * np.pi / 4:
            tdiff = tmod - np.pi / 2
            xf = -((u - l) / 2) * (np.cos(4*tdiff)-1) + l
            yf = u
        # up
        elif tmod >= 5 * np.pi / 4 and tmod <= 7 * np.pi / 4:
            tdiff = tmod - 3 * np.pi / 2
            xf = -((u - l) / 2) * (np.cos(4*tdiff)-1) + l
            yf = u
        # left
        elif tmod >= 3 * np.pi / 4 and tmod <= 5 * np.pi / 4:
            tdiff = tmod - np.pi
            xf = u
            yf = -((u - l) / 2) * (np.cos(4*tdiff)-1) + l
        # right
        else:
            tdiff = tmod
            xf = u
            yf = -((u - l) / 2) * (np.cos(4*tdiff)-1) + l
        
        xadj = xf * newScale
        yadj = yf * newScale

        x += xadj
        y += yadj

        # if x > 0 and x < dimX and y > 0 and y < dimY:
        #     pygame.draw.circle(screen, "green", (x, y), 3, 3)


        newAngle = -1 * (t - upTheta)
        # could adjust angle to match spiral instead of a circle
        
        text = pygame.transform.rotozoom(text, np.degrees(newAngle), (r / baseRad) / 10)
        
        screen.blit(text, (x, y))

        r *= radStep
        t += thetaStep

    if debug:
        pygame.draw.line(screen, "red", (0, dimY/2), (dimX, dimY/2), 1)
        pygame.draw.line(screen, "red", (dimX/2, 0), (dimX/2, dimY), 1)
        # diag
        pygame.draw.line(screen, "red", (0, 0), (dimX, dimY), 1)
        pygame.draw.line(screen, "red", (dimX, 0), (0, dimY), 1)

    pygame.display.flip()
    clock.tick(frameRate)  # limits FPS to 60

pygame.quit()

In [5]:
# threaded
from queue import Queue
import threading

debug = False
pygame.init()
dimX = 1920
dimY = 1080
screen = pygame.display.set_mode((dimX, dimY))
clock = pygame.time.Clock()
running = True
font = pygame.font.SysFont("Consolas", 200)

upTheta = 3 * np.pi / 2
baseRad = 75

radius = baseRad * 2
theta = upTheta

radStep = 1.007
stepsPerCircle = 40
thetaStep = 2 * np.pi / stepsPerCircle

frameRate = 144

scaleSpeed = 1/30
rotSpeed = 1/3

drawQ = Queue()

def drawThread(queue_t):
    global screen, baseRad
    while True:
        line = queue_t.get(block=True)
        if line == "end":
            return
        x, y, newAngle, r, text = line
        text = pygame.transform.rotozoom(text, np.degrees(newAngle), (r / baseRad) / 10)
        screen.blit(text, (x, y))
        queue_t.task_done()

numThreads = 16
threads = []
for i in range(numThreads):
    t = threading.Thread(target=drawThread, args=(drawQ,))
    t.daemon = True
    t.start()
    threads.append(t)

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    screen.fill("white")

    keys = pygame.key.get_pressed()

    fps = clock.get_fps()
    if debug:
        print(fps)
    if fps == 0:
        fps = 15
    if keys[pygame.K_UP]:
        radius *= radStep ** ((stepsPerCircle * scaleSpeed) / (fps/15))
    if keys[pygame.K_DOWN]:
        radius /= radStep ** ((stepsPerCircle * scaleSpeed) / (fps/15))
    if keys[pygame.K_LEFT]:
        theta += thetaStep * (rotSpeed / (fps/15))
        radius *= radStep ** (rotSpeed / (fps/15))
    if keys[pygame.K_RIGHT]:
        theta -= thetaStep * (rotSpeed / (fps/15))
        radius /= radStep ** (rotSpeed / (fps/15))

    if keys[pygame.K_w]:
        scaleSpeed *= 1.1
    if keys[pygame.K_s]:
        scaleSpeed /= 1.1
    if keys[pygame.K_a]:
        rotSpeed *= 1.1
    if keys[pygame.K_d]:
        rotSpeed /= 1.1

    titleText = font.render(title, True, "black")
    titleText = pygame.transform.scale_by(titleText, (radius * 1.75)/titleText.get_width())
    titleRect = titleText.get_rect()
    titleRect.center = (dimX/2, dimY/2)
    screen.blit(titleText, titleRect)
    
    r, t = radius, theta
    for i in range(len(testStr)):
        if r < baseRad * (2/3): # significantly improves perf from 1/3 to 2/3
            r *= radStep
            t += thetaStep
            continue
        if r > np.sqrt((dimX/2)**2 + (dimY/2)**2):
            break
        text = font.render(testStr[i], True, "black")
        x, y = polarToCart(r, t)
        
        newScale = 200 * (r / baseRad) / 10
        x += dimX/2 
        y += dimY/2

        # draw dot at x, y
        if debug:
            if x > 0 and x < dimX and y > 0 and y < dimY:
                pygame.draw.circle(screen, "red", (x, y), 3, 3)

        # left right -0.5, -0.3
        # up down -0.3 -0.5
        # both diag -0.55, -0.55
        
        # this is to adjust the location of the rotated text to match the spiral
        # when adding the title copilot just suggested to center the rectangle, so didn't need all of this but leaving it anyways
        u = -0.5
        l = -0.28
        tmod = mod2Pi(t)
        # down
        if tmod >= np.pi / 4 and tmod <= 3 * np.pi / 4:
            tdiff = tmod - np.pi / 2
            xf = -((u - l) / 2) * (np.cos(4*tdiff)-1) + l
            yf = u
        # up
        elif tmod >= 5 * np.pi / 4 and tmod <= 7 * np.pi / 4:
            tdiff = tmod - 3 * np.pi / 2
            xf = -((u - l) / 2) * (np.cos(4*tdiff)-1) + l
            yf = u
        # left
        elif tmod >= 3 * np.pi / 4 and tmod <= 5 * np.pi / 4:
            tdiff = tmod - np.pi
            xf = u
            yf = -((u - l) / 2) * (np.cos(4*tdiff)-1) + l
        # right
        else:
            tdiff = tmod
            xf = u
            yf = -((u - l) / 2) * (np.cos(4*tdiff)-1) + l
        
        xadj = xf * newScale
        yadj = yf * newScale

        x += xadj
        y += yadj

        # if x > 0 and x < dimX and y > 0 and y < dimY:
        #     pygame.draw.circle(screen, "green", (x, y), 3, 3)


        newAngle = -1 * (t - upTheta)
        
        # text = pygame.transform.rotozoom(text, np.degrees(newAngle), (r / baseRad) / 10)
        # screen.blit(text, (x, y))
        drawQ.put((x, y, newAngle, r, text))

        r *= radStep
        t += thetaStep

    drawQ.join()

    if debug:
        pygame.draw.line(screen, "red", (0, dimY/2), (dimX, dimY/2), 1)
        pygame.draw.line(screen, "red", (dimX/2, 0), (dimX/2, dimY), 1)
        # diag
        pygame.draw.line(screen, "red", (0, 0), (dimX, dimY), 1)
        pygame.draw.line(screen, "red", (dimX, 0), (0, dimY), 1)

    pygame.display.flip()
    clock.tick(frameRate)  # limits FPS to 60

for _ in range(numThreads):
    drawQ.put("end")

pygame.quit()